## Fix for ScraperAgent Not Defined

If you get `NameError: name 'ScraperAgent' is not defined`, make sure to run the cell below **before** running the main test cell. This will define a minimal ScraperAgent in the notebook scope.

In [1]:
# Minimal ScraperAgent for notebook testing
class ScraperAgent:
    def scrape(self, data):
        return data

## Update Python Libraries (If Needed)

If you're experiencing issues with newer API keys or models, you might need to update the dependencies.

In [2]:
# Uncomment and run this cell if you need to update libraries
# !pip install -U openai langchain langchain-openai

## Load Configuration

Important: Run this cell first to load the configuration file.

In [3]:
import json

# Load configuration from config.json
try:
    with open('config.json', 'r') as f:
        config = json.load(f)
    print(f"Configuration loaded successfully.")
    print(f"Model: {config.get('model_name', 'Not specified')}")
    api_key = config.get('api_key', '')
    print(f"API key length: {len(api_key)}")
    if len(api_key) > 20:
        print(f"API key starts with: {api_key[:10]}...")
    else:
        print("Warning: API key seems too short or empty.")
except Exception as e:
    print(f"Error loading configuration: {str(e)}")
    # Create a default config to avoid errors
    config = {"model_name": "o3-mini", "api_key": ""}

Configuration loaded successfully.
Model: o3-mini
API key length: 164
API key starts with: sk-proj-PG...


## Run the Multi-Agent Structure Test

This test uses real sample review data from sample_reviews.py.

In [4]:
import os
# from agents.scraper import ScraperAgent  # Now defined above
from agents.preprocessing import PreprocessingAgent
from agents.analyzer import AnalyzerAgent
from agents.coordinator import CoordinatorAgent
from agents.report import ReportAgent
from agents.gpt_handler import GPTHandler
from data.sample_reviews import SHOPEE_REVIEWS, YOUTUBE_REVIEWS

# Set OpenAI API key environment variable (needed for langchain)
os.environ['OPENAI_API_KEY'] = config.get('api_key', '')

# Verify CoordinatorAgent accepts config parameter
import inspect
coord_params = inspect.signature(CoordinatorAgent.__init__).parameters
if 'config' not in coord_params:
    print("WARNING: CoordinatorAgent doesn't have config parameter. Module reload needed.")
    print("Available parameters:", list(coord_params.keys()))
else:
    print("CoordinatorAgent correctly accepts config parameter")

# Use actual sample review data instead of hardcoded test data
test_reviews = SHOPEE_REVIEWS + YOUTUBE_REVIEWS
print(f"Loaded {len(SHOPEE_REVIEWS)} Shopee reviews and {len(YOUTUBE_REVIEWS)} YouTube reviews")

# Step 1: Scrape (pass-through for sample data)
scraper = ScraperAgent()
scraped = scraper.scrape(test_reviews)
print(f"Scraped {len(scraped)} reviews.")

# Step 2: Preprocess
preprocessor = PreprocessingAgent()
preprocessed = preprocessor.preprocess(scraped)
print(f"Preprocessed {len(preprocessed)} reviews.")

# Display a few examples of preprocessed reviews
for i, review in enumerate(preprocessed[:3]):
    print(f"\nExample {i+1}:")
    if 'review_text' in review:
        print(f"- Text: {review['review_text'][:50]}...")
    elif 'comment' in review:
        print(f"- Comment: {review['comment'][:50]}...")
    print(f"- Platform: {review.get('platform', 'Unknown')}")
    print(f"- Product: {review.get('product_name', 'Unknown')}")

# Step 3: Analyze (single agent test)
analyzer = AnalyzerAgent(config=config)
# Only analyze a few examples to avoid long execution times
analysis_sample = preprocessed[:3]
analysis_results = [analyzer.analyze(r) for r in analysis_sample]
for res in analysis_results:
    print(res)

try:
    # Step 4: Coordinator (should use config)
    coordinator = CoordinatorAgent(config=config)
    product_id = "S001"  # Wireless Earbuds
    print(f"\nAnalyzing reviews for product ID: {product_id} (Wireless Earbuds)")
    product_reviews = [r for r in preprocessed if r["product_id"] == product_id]
    print(f"Found {len(product_reviews)} reviews for this product")
    
    # Use only a small subset to speed up testing
    test_subset = product_reviews[:3]
    print(f"Using a subset of {len(test_subset)} reviews for testing")
    
    summary = coordinator.analyze_product_reviews(test_subset)
    print("Coordinator summary:", summary)
    
    # Step 5: Report Agent
    report_agent = ReportAgent()
    report = report_agent.compile(summary)
    print("\nReport:")
    for key, value in report['products'][0].items():
        if isinstance(value, list) and len(value) > 3:
            print(f"{key}: {value[:3]}...")
        else:
            print(f"{key}: {value}")
    
    # Step 6: GPT Handler
    gpt_handler = GPTHandler(config=config)
    gpt_summary = gpt_handler.generate_summary(report)
    print("\nGPT Summary:", gpt_summary)
    
except Exception as e:
    print(f"Error during execution: {str(e)}")
    print("\nIf this is an authentication error, please check that:")
    print("1. Your API key in config.json is correct and active")
    print("2. You have billing set up in your OpenAI account")
    print("3. The model name in your config.json is available to your account")
    
    import traceback
    traceback.print_exc()

CoordinatorAgent correctly accepts config parameter
Loaded 5 Shopee reviews and 5 YouTube reviews
Scraped 10 reviews.
Preprocessed 10 reviews.

Example 1:
- Text: extremely satisfied with these earbuds! the sound ...
- Platform: Shopee
- Product: Wireless Earbuds

Example 2:
- Text: disappointed with these earbuds. they disconnect f...
- Platform: Shopee
- Product: Wireless Earbuds

Example 3:
- Text: average product. sound quality is okay but not as ...
- Platform: Shopee
- Product: Wireless Earbuds
{'agent': 'AnalyzerAgent', 'sentiment': 'negative', 'intensity': 'moderate', 'key_phrases': ['extremely', 'satisfied', 'these'], 'text': 'extremely satisfied with these earbuds! the sound quality is amazing and battery life is impressive. worth every penny.'}
{'agent': 'AnalyzerAgent', 'sentiment': 'positive', 'intensity': 'moderate', 'key_phrases': ['disappointed', 'these', 'earbuds.'], 'text': 'disappointed with these earbuds. they disconnect frequently and the battery drains too quickly